<a href="https://colab.research.google.com/github/jazoza/mad/blob/main/04_MAD_SOM_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Arguments with Data

Experiments with clustering and projecting

In [ ]:
# third final workshop day
# the most intersting notebook :)